# Полный Data Science

В этом ноутбуке мы решаем задачу **прогноза оттока клиентов (churn)** по табличным данным.

Будем работать с датасетом наподобие *Telco Customer Churn*:
- каждая строка — клиент телеком/подписочного сервиса,
- признаки — характеристики клиента и его поведения (тариф, услуги, длительность, платежи...),
- целевая переменная `Churn` — ушёл клиент (`Yes`) или остался (`No`).

В рамках задания мы:
1. Сформулируем бизнес-задачу и выберем метрики качества.
2. Сделаем EDA и предобработку данных.
3. Сгенерируем и отфильтруем признаки.
4. Разобьём данные на train/val/test.
5. Обучим и сравним модели:
   - KNN,
   - Logistic Regression,
   - DecisionTree,
   - RandomForest,
   - LightGBM (градиентный бустинг).
6. Подберём гиперпараметры (GridSearchCV) и оценим влияние на качество.
7. Сделаем отбор признаков под конкретные модели.
8. Протестируете лучшую(ие) модель(и) на test.
9. Попробуем простой стэкинг моделей.


In [ ]:
# Базовые импорты
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier

from lightgbm import LGBMClassifier

%matplotlib inline
sns.set(style="whitegrid", palette="muted", font_scale=1.1)

RANDOM_STATE = 42

## 1. Загрузка данных и описание бизнес-задачи

### Бизнес-задача

Мы — менеджеры по продукту/маркетингу в подписочном сервисе (телеком, SaaS, онлайн-сервис).

Наша цель — **прогнозировать отток клиентов (churn)**, чтобы:
- заранее выделять "группу риска" и запускать кампании удержания,
- оптимизировать бюджет на retention-активности,
- оценивать влияние изменений продукта/тарифов на отток.

### Выбор метрик качества

Задача — бинарная классификация (`Churn = Yes/No`), при этом класс `Yes` (ушёл) обычно **редкий**.


Какую метрику классификации будем использовать?

In [ ]:
# Загрузка датасета (путь при необходимости поменяйте)
df = pd.read_csv("telco_churn.csv")

df.head()

Нас интересует:
- **Recall** по классу `Churn=Yes` — какую долю действительно уходящих клиентов мы ловим? (важно не пропустить тех, кого можно спасти)
- **Precision** по `Churn=Yes` — какой процент среди отобранных нами "рисковых" клиентов реально уйдут? (важно не тратить бюджет на удержание тех, кто и так останется)
- **F1-score** по `Churn=Yes` — баланс между Precision и Recall,
- **ROC-AUC** — общая способность модели разделять уходящих и остающихся клиентов, независимо от порога.

Дополнительно можем смотреть **accuracy**, но при сильном дисбалансе классов она менее информативна.

In [ ]:
# Общая информация о данных
display(df.info())
display(df.describe(include="all"))

## 2. EDA

Проверим:
- распределение целевой переменной (баланс классов),
- основные числовые признаки,
- связи некоторых признаков с оттоком.

In [ ]:
# Распределение целевой переменной (Churn)
plt.figure(figsize=(4,4))
df["Churn"].value_counts().plot(kind="bar")
plt.title("Распределение классов Churn")
plt.ylabel("Количество клиентов")
plt.show()

df["Churn"].value_counts(normalize=True)

In [ ]:
# Пример: распределение ежемесячных платежей для ушедших/оставшихся
plt.figure(figsize=(6,4))
sns.histplot(data=df, x="MonthlyCharges", hue="Churn", bins=30, kde=False, multiple="stack")
plt.title("MonthlyCharges по классам Churn")
plt.show()

# Пример: завимость оттока от типа контракта
plt.figure(figsize=(6,4))
churn_by_contract = pd.crosstab(df["Contract"], df["Churn"], normalize="index")["Yes"].sort_values(ascending=False)
churn_by_contract.plot(kind="bar")
plt.title("Доля оттока по типу контракта")
plt.ylabel("Доля Churn=Yes")
plt.show()

In [ ]:
# Пример: корреляции между числовыми признаками и оттоком
num_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
corr = df[num_cols + ["Churn"]].copy()

# Переведем Churn в 0/1 для корреляций
corr["Churn"] = (df["Churn"] == "Yes").astype(int)
corr_matrix = corr.corr()

plt.figure(figsize=(6,5))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Корреляция числовых признаков и Churn")
plt.show()

## 3. Предобработка данных

Типичные шаги:
- удалить или обработать идентификаторы (например, `customerID`),
- привести строковые числовые признаки к числовому типу (`TotalCharges` часто приходит как object),
- обработать пропуски,
- разделить признаки на числовые и категориальные.

In [ ]:
# Удалим идентификатор, если он есть
if "customerID" in df.columns:
    df = df.drop(columns=["customerID"])

# Преобразуем TotalCharges в число, если это object
if "TotalCharges" in df.columns and df["TotalCharges"].dtype == "object":
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"].replace(" ", np.nan), errors="coerce")

# Посмотрим на пропуски
df.isna().mean().sort_values(ascending=False).head(10)

In [ ]:
# Простая обработка пропусков:
# - числовые: медиана
# - категориальные: мода (самое частое значение)

num_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()

for col in num_cols:
    median = df[col].median()
    df[col] = df[col].fillna(median)

for col in cat_cols:
    mode = df[col].mode()[0]
    df[col] = df[col].fillna(mode)

df.isna().sum().sum()  # проверим, что пропусков больше нет

## 4. Генерация и первичная фильтрация признаков

Примеры новых фич для задачи оттока:
- `AvgCharges` = `TotalCharges` / `tenure` — средний чек клиента за месяц,
- `IsLongTermContract` — флаг долгосрочного контракта (1/2 года),
- биннинг `tenure` по группам.

После генерации можем удалить "подозрительно" дублирующие признаки и те, что сложно использовать (например, почти константы).

In [ ]:
# Сгенерируем несколько новых признаков

if "tenure" in df.columns and "TotalCharges" in df.columns:
    df["AvgCharges"] = df["TotalCharges"] / df["tenure"].replace(0, np.nan)
    df["AvgCharges"] = df["AvgCharges"].fillna(df["MonthlyCharges"])  # если tenure=0, используем MonthlyCharges

# Флаг долгосрочного контракта
if "Contract" in df.columns:
    df["IsLongTermContract"] = df["Contract"].isin(["One year", "Two year"]).astype(int)

# Биннинг tenure
if "tenure" in df.columns:
    df["TenureGroup"] = pd.cut(
        df["tenure"],
        bins=[0, 12, 24, 48, 60, np.inf],
        labels=["0-12", "12-24", "24-48", "48-60", "60+"],
        include_lowest=True,
    )

df.head()

In [ ]:
# Обновим списки числовых и категориальных признаков
num_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()

print("Числовые признаки:", num_cols)
print("Категориальные признаки:", cat_cols)

Для первичной фильтрации можно:
- убрать признаки с почти одной категорией/одним значением (low variance - маленькой дисперсией),
- убрать очевидные дублирующие признаки.

Здесь оставим всё, кроме таргета (`Churn`).

In [ ]:
target_col = "Churn"

X = df.drop(columns=[target_col])
y = (df[target_col] == "Yes").astype(int)

X.shape, y.mean()  # размер и доля оттока

## 5. Разбиение на train / val / test

Сделаем разбиение 60% / 20% / 20% с учётом стратификации (сохранение доли таргета в каждом датасете) по оттоку.


Мы уже знаем, зачем нужны Train/Test. Зачем же нужна val выборка? Ваши варианты?

In [ ]:
# Сначала отделим test 20%
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

# Теперь из оставшихся 80% выделим 20% под val → получится 60/20/20
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=RANDOM_STATE, stratify=y_temp
)

print("Train:", X_train.shape, y_train.mean())
print("Val:  ", X_val.shape, y_val.mean())
print("Test: ", X_test.shape, y_test.mean())

Val (валидационная выборка, выборка для отладки) -  нужна для подбора гиперпараметров модели. Это настройки алгоритма, которые не выучиваются из данных напрямую (например, количество деревьев в случайном лесу).


Тут наша основная задача: не подсматривать в тестовую выборку в процессе улучшения модели (по тестовой выборке уже финально принимаем решение по модели). Иначе - можно бесконечно подсматривать в тестовые данные и под них улучшать/подгонять модель, что может приводить к переобучению и на новых данных будет заметная потеря качества.
При этом подбирать гиперпараметры на тренировочных данных - тоже плохо. Так как на тренировочных данных мы не можем ничего сказать про модель (по крайней мере, ничего хорошего)

## 6. Кодирование категориальных признаков и масштабирование числовых

Сделаем one-hot кодирование категориальных признаков и масштабирование числовых.
Чтобы упростить, используем `pd.get_dummies` и `StandardScaler` только для числовых колонок.

In [ ]:
# One-hot кодирование категориальных признаков
X_train_enc = pd.get_dummies(X_train, drop_first=True)
X_val_enc = pd.get_dummies(X_val, drop_first=True)
X_test_enc = pd.get_dummies(X_test, drop_first=True)

# Выровняем набор колонок (после get_dummies они могут отличаться)
X_train_enc, X_val_enc = X_train_enc.align(X_val_enc, join="left", axis=1, fill_value=0)
X_train_enc, X_test_enc = X_train_enc.align(X_test_enc, join="left", axis=1, fill_value=0)

X_train_enc.shape, X_val_enc.shape, X_test_enc.shape

In [ ]:
# Масштабируем числовые признаки (важно для KNN и логрегрессии)

num_mask = X_train_enc.columns.str.contains("MonthlyCharges") | X_train_enc.columns.str.contains("TotalCharges") | X_train_enc.columns.str.contains("tenure") | X_train_enc.columns.str.contains("AvgCharges")

scaler = StandardScaler()
X_train_scaled = X_train_enc.copy()
X_val_scaled = X_val_enc.copy()
X_test_scaled = X_test_enc.copy()

cols_to_scale = X_train_enc.columns[num_mask]

X_train_scaled[cols_to_scale] = scaler.fit_transform(X_train_enc[cols_to_scale])
X_val_scaled[cols_to_scale] = scaler.transform(X_val_enc[cols_to_scale])
X_test_scaled[cols_to_scale] = scaler.transform(X_test_enc[cols_to_scale])

X_train_scaled.head()

## 7. Базовое обучение моделей и измерение времени

Обучим базовые версии моделей:
- KNN,
- Logistic Regression,
- DecisionTree,
- RandomForest,
- LightGBM.

Для каждой модели измерим:
- время обучения,
- время предсказания на валидации,
- ключевые метрики (accuracy, precision, recall, F1, ROC-AUC).

In [ ]:
def evaluate_model(name, model, X_tr, y_tr, X_val, y_val):
    print(f"===== {name} =====")
    t0 = time.time()
    model.fit(X_tr, y_tr)
    t_train = time.time() - t0

    t0 = time.time()
    y_val_pred = model.predict(X_val)
    t_pred = time.time() - t0

    if hasattr(model, "predict_proba"):
        y_val_proba = model.predict_proba(X_val)[:, 1]
        roc_auc = roc_auc_score(y_val, y_val_proba)
    else:
        roc_auc = np.nan

    acc = accuracy_score(y_val, y_val_pred)
    prec = precision_score(y_val, y_val_pred, zero_division=0)
    rec = recall_score(y_val, y_val_pred, zero_division=0)
    f1 = f1_score(y_val, y_val_pred, zero_division=0)

    print(f"Время обучения:  {t_train:.4f} c")
    print(f"Время предсказания: {t_pred:.4f} c")
    print(f"Accuracy:  {acc:.3f}")
    print(f"Precision: {prec:.3f}")
    print(f"Recall:    {rec:.3f}")
    print(f"F1:        {f1:.3f}")
    print(f"ROC-AUC:   {roc_auc:.3f}")
    print()

    return {
        "name": name,
        "model": model,
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "roc_auc": roc_auc,
        "t_train": t_train,
        "t_pred": t_pred,
    }
results = [] # сюда будем записывать результаты работы моделей

In [ ]:
# KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn_res = evaluate_model("KNN", knn, X_train_scaled, y_train, X_val_scaled, y_val)
results.append(knn_res)

In [ ]:
# Logistic Regression
log_reg = LogisticRegression(max_iter=200, C=1.0, random_state=42)
log_reg_res = evaluate_model("LogisticRegression", log_reg, X_train_scaled, y_train, X_val_scaled, y_val)
results.append(log_reg_res)

In [ ]:
# Decision Tree
dt = DecisionTreeClassifier(max_depth=5, random_state=42)
results.append(evaluate_model("DecisionTree", dt, X_train_enc, y_train, X_val_enc, y_val))

In [ ]:
# Random Forest
rf = RandomForestClassifier(n_estimators=100, max_depth=7, random_state=42, n_jobs=-1)
results.append(evaluate_model("RandomForest", rf, X_train_enc, y_train, X_val_enc, y_val))

In [ ]:
# LightGBM
lgbm = LGBMClassifier(
    n_estimators=200,
    max_depth=-1,
    learning_rate=0.05,
    random_state=42,
    n_jobs=-1,
)
results.append(evaluate_model("LightGBM", lgbm, X_train_enc, y_train, X_val_enc, y_val))

In [ ]:
pd.DataFrame(results)[["name", "accuracy", "precision", "recall", "f1", "roc_auc", "t_train", "t_pred"]]

## 8. Подбор гиперпараметров (GridSearchCV)

Сделаем небольшие гриды, чтобы не ждать слишком долго, и нарисуем графики зависимости качества от ключевого гиперпараметра.

### 8.1. KNN — подбираем `n_neighbors`

In [ ]:
k_values = [3, 5, 7, 9, 11, 15, 20, 25, 30, 35, 40, 50]
knn_scores = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    y_val_pred = knn.predict(X_val_scaled)
    f1 = f1_score(y_val, y_val_pred)
    knn_scores.append(f1)

plt.figure(figsize=(6,4))
plt.plot(k_values, knn_scores, marker="o")
plt.title("KNN: F1-score на валидации в зависимости от k")
plt.xlabel("k (число соседей)")
plt.ylabel("F1-score")
plt.show()

best_k = k_values[int(np.argmax(knn_scores))]
best_k

### 8.2. DecisionTree — подбираем `max_depth`

In [ ]:
depth_values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, None]
dt_scores = []

for d in depth_values:
    dt = DecisionTreeClassifier(max_depth=d, random_state=RANDOM_STATE)
    dt.fit(X_train_enc, y_train)
    y_val_pred = dt.predict(X_val_enc)
    f1 = f1_score(y_val, y_val_pred)
    dt_scores.append(f1)

plt.figure(figsize=(6,4))
labels = [str(d) for d in depth_values]
plt.plot(labels, dt_scores, marker="o")
plt.title("DecisionTree: F1-score на валидации в зависимости от max_depth")
plt.xlabel("max_depth")
plt.ylabel("F1-score")
plt.show()

depth_values[int(np.argmax(dt_scores))]

### 8.3. RandomForest — подбираем `n_estimators` и `max_depth` (через GridSearchCV)

In [ ]:
param_grid_rf = {
    "n_estimators": [50, 75, 100, 125, 150, 175, 200, 250],
    "max_depth": [2, 3, 5, 7, 9],
}

rf_base = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1)

grid_rf = GridSearchCV(
    rf_base,
    param_grid_rf,
    scoring="f1",
    cv=3,
    n_jobs=-1,
    verbose=1,
)
grid_rf.fit(X_train_enc, y_train)

print("Лучшие параметры RF:", grid_rf.best_params_)
print("Лучший F1 (cv):", grid_rf.best_score_)

### 8.4. LightGBM — подбираем `n_estimators` и `num_leaves` (GridSearchCV)

In [ ]:
param_grid_lgbm = {
    "n_estimators": [100, 200],
    "num_leaves": [15, 31, 63],
}

lgbm_base = LGBMClassifier(
    learning_rate=0.05,
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

grid_lgbm = GridSearchCV(
    lgbm_base,
    param_grid_lgbm,
    scoring="f1",
    cv=3,
    n_jobs=-1,
    verbose=1,
)
grid_lgbm.fit(X_train_enc, y_train)

print("Лучшие параметры LGBM:", grid_lgbm.best_params_)
print("Лучший F1 (cv):", grid_lgbm.best_score_)

### 8.5. Logistic Regression — подбор `C` (силы регуляризации)

In [ ]:
param_grid_log = {"C": [0.1, 0.5, 1.0, 2.0, 5.0]}

log_base = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)

grid_log = GridSearchCV(
    log_base,
    param_grid_log,
    scoring="f1",
    cv=3,
    n_jobs=-1,
)
grid_log.fit(X_train_scaled, y_train)

print("Лучшие параметры LogReg:", grid_log.best_params_)
print("Лучший F1 (cv):", grid_log.best_score_)

## 9. Отбор признаков под конкретные модели

Для деревьев, лесов и бустинга можно использовать **важность признаков (feature importance)**.

Сделаем, например, отбор топ-20 признаков для LightGBM и RandomForest, и обучим модели только на них.

In [ ]:
# Обучим лучшие модели RF и LGBM на всех признаках
best_rf = grid_rf.best_estimator_
best_rf.fit(X_train_enc, y_train)

best_lgbm = grid_lgbm.best_estimator_
best_lgbm.fit(X_train_enc, y_train)

# Возьмем важности признаков из LGBM
feat_imp = pd.Series(best_lgbm.feature_importances_, index=X_train_enc.columns)
feat_imp = feat_imp.sort_values(ascending=False)

plt.figure(figsize=(7,5))
feat_imp.head(20).plot(kind="bar")
plt.title("LightGBM: топ-20 признаков по важности")
plt.tight_layout()
plt.show()

top_features = feat_imp.head(20).index.tolist()
len(top_features), top_features[:5]

In [ ]:
# Сформируем выборки только с топ-20 признаков
X_train_top = X_train_enc[top_features]
X_val_top = X_val_enc[top_features]
X_test_top = X_test_enc[top_features]

# Переобучим LGBM и RF на топ-20
best_rf_top = grid_rf.best_estimator_
best_rf_top.fit(X_train_top, y_train)
y_val_pred_rf_top = best_rf_top.predict(X_val_top)
f1_rf_top = f1_score(y_val, y_val_pred_rf_top)

best_lgbm_top = grid_lgbm.best_estimator_
best_lgbm_top.fit(X_train_top, y_train)
y_val_pred_lgbm_top = best_lgbm_top.predict(X_val_top)
f1_lgbm_top = f1_score(y_val, y_val_pred_lgbm_top)

print("F1 RF (топ-20 признаков):", f1_rf_top)
print("F1 LGBM (топ-20 признаков):", f1_lgbm_top)

## 10. Финальное тестирование на `test`

Выберем 1–2 лучшие модели по F1/ROC-AUC на валидации и протестируем их на `test`.

Здесь в качестве примера возьмём:
- лучшую логистическую регрессию (grid_log.best_estimator_),
- LightGBM на топ-20 признаков.

In [ ]:
# Лучшая логистическая регрессия
final_log = grid_log.best_estimator_
final_log.fit(X_train_scaled, y_train)

y_test_pred_log = final_log.predict(X_test_scaled)
y_test_proba_log = final_log.predict_proba(X_test_scaled)[:, 1]

print("=== LogisticRegression (test) ===")
print("Accuracy:", accuracy_score(y_test, y_test_pred_log))
print("Precision:", precision_score(y_test, y_test_pred_log))
print("Recall:", recall_score(y_test, y_test_pred_log))
print("F1:", f1_score(y_test, y_test_pred_log))
print("ROC-AUC:", roc_auc_score(y_test, y_test_proba_log))

cm_log = confusion_matrix(y_test, y_test_pred_log)
sns.heatmap(cm_log, annot=True, fmt="d", cmap="Blues")
plt.title("LogReg: матрица ошибок (test)")
plt.xlabel("Предсказанный класс")
plt.ylabel("Истинный класс")
plt.show()

In [ ]:
# Лучший LightGBM на топ-20 признаков
final_lgbm = best_lgbm_top

y_test_pred_lgbm = final_lgbm.predict(X_test_top)
y_test_proba_lgbm = final_lgbm.predict_proba(X_test_top)[:, 1]

print("=== LightGBM (test, топ-20 признаков) ===")
print("Accuracy:", accuracy_score(y_test, y_test_pred_lgbm))
print("Precision:", precision_score(y_test, y_test_pred_lgbm))
print("Recall:", recall_score(y_test, y_test_pred_lgbm))
print("F1:", f1_score(y_test, y_test_pred_lgbm))
print("ROC-AUC:", roc_auc_score(y_test, y_test_proba_lgbm))

cm_lgbm = confusion_matrix(y_test, y_test_pred_lgbm)
sns.heatmap(cm_lgbm, annot=True, fmt="d", cmap="Greens")
plt.title("LightGBM: матрица ошибок (test)")
plt.xlabel("Предсказанный класс")
plt.ylabel("Истинный класс")
plt.show()

## 11. Попытка стэкинга (Stacking)

Скомбинируем несколько моделей (например, логистическую регрессию, RandomForest и LightGBM) в **стэкинг**:
- базовые модели дают свои прогнозы,
- метамодель (ещё одна логистическая регрессия) учится комбинировать их предсказания.

In [ ]:
base_estimators = [
    ("logreg", grid_log.best_estimator_),
    ("rf", grid_rf.best_estimator_),
    ("lgbm", grid_lgbm.best_estimator_),
]

stack_clf = StackingClassifier(
    estimators=base_estimators,
    final_estimator=LogisticRegression(max_iter=1000, random_state=RANDOM_STATE),
    n_jobs=-1,
)

# Для простоты используем enc-признаки (без урезания до топ-20)
stack_clf.fit(X_train_enc, y_train)

y_val_pred_stack = stack_clf.predict(X_val_enc)
y_val_proba_stack = stack_clf.predict_proba(X_val_enc)[:, 1]

print("=== Stacking (val) ===")
print("Accuracy:", accuracy_score(y_val, y_val_pred_stack))
print("Precision:", precision_score(y_val, y_val_pred_stack))
print("Recall:", recall_score(y_val, y_val_pred_stack))
print("F1:", f1_score(y_val, y_val_pred_stack))
print("ROC-AUC:", roc_auc_score(y_val, y_val_proba_stack))

In [ ]:
# Оценим стэкинг на test
y_test_pred_stack = stack_clf.predict(X_test_enc)
y_test_proba_stack = stack_clf.predict_proba(X_test_enc)[:, 1]

print("=== Stacking (test) ===")
print("Accuracy:", accuracy_score(y_test, y_test_pred_stack))
print("Precision:", precision_score(y_test, y_test_pred_stack))
print("Recall:", recall_score(y_test, y_test_pred_stack))
print("F1:", f1_score(y_test, y_test_pred_stack))
print("ROC-AUC:", roc_auc_score(y_test, y_test_proba_stack))

cm_stack = confusion_matrix(y_test, y_test_pred_stack)
sns.heatmap(cm_stack, annot=True, fmt="d", cmap="Purples")
plt.title("Stacking: матрица ошибок (test)")
plt.xlabel("Предсказанный класс")
plt.ylabel("Истинный класс")
plt.show()

Если более простая модель (как тут - логистическая регрессия) дает результат не сильно хуже продвинутых моделей, то не нужно усложнять.

Это плюс к:
- Стабильности модели
- Простоте внедрения модели
- Возможности интерпретации результатов

## Заключение

- Мы сравнили несколько моделей от простых (логистическая регрессия) до более сложных (лес, бустинг, стэкинг).
- Оценивали их по метрикам, важным для бизнеса: Recall/Precision/F1, ROC-AUC.
- Посмотрели на важность признаков и влияние отбора фич.
- Собрали ансамбль (стэкинг) и проверили, даёт ли он выигрыш на тестовой выборке.

Дальше можно экспериментировать:
- менять метрики оптимизации (например, на ROC-AUC или PR-AUC),
- тестировать другие модели,
- внедрять cost-sensitive подход (учитывать стоимость ошибок отдельно для FP и FN).